<a href="https://colab.research.google.com/github/ArimaValanImmanuel/P3DL/blob/main/Pose3dLifting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow 1.13.x

and other necessary libraries

In [ ]:
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.13.1

In [ ]:
!pip freeze | grep -i -e tensorflow

In [ ]:
!pip uninstall imgaug
!pip install imgaug==0.2.6
!pip install -q -U opencv-python

# Package/Lifting/Utils

1.config
2.cpm
3.draw
4.prob_model
5.process
6.upright_fast

#Package/Lifting

_pose_estimator


config

In [ ]:
__all__ = [
    'VISIBLE_PART',
    'MIN_NUM_JOINTS',
    'CENTER_TR',
    'SIGMA',
    'STRIDE',
    'SIGMA_CENTER',
    'INPUT_SIZE',
    'OUTPUT_SIZE',
    'NUM_JOINTS',
    'NUM_OUTPUT',
    'H36M_NUM_JOINTS',
    'JOINT_DRAW_SIZE',
    'LIMB_DRAW_SIZE'
]

# threshold
VISIBLE_PART = 1e-3
MIN_NUM_JOINTS = 5
CENTER_TR = 0.4

# net attributes
SIGMA = 7
STRIDE = 8
SIGMA_CENTER = 21
INPUT_SIZE = 368
OUTPUT_SIZE = 46
NUM_JOINTS = 14
NUM_OUTPUT = NUM_JOINTS + 1
H36M_NUM_JOINTS = 17

# draw options
JOINT_DRAW_SIZE = 3
LIMB_DRAW_SIZE = 2
NORMALISATION_COEFFICIENT = 1280*720

# test options
BATCH_SIZE = 4


cpm

In [ ]:
import tensorflow as tf
import tensorflow.contrib.layers as layers

__all__ = [
    'inference_person',
    'inference_pose'
]


def inference_person(image):
    with tf.variable_scope('PersonNet'):
        conv1_1 = layers.conv2d(
            image, 64, 3, 1, activation_fn=None, scope='conv1_1')
        conv1_1 = tf.nn.relu(conv1_1)
        conv1_2 = layers.conv2d(
            conv1_1, 64, 3, 1, activation_fn=None, scope='conv1_2')
        conv1_2 = tf.nn.relu(conv1_2)
        pool1_stage1 = layers.max_pool2d(conv1_2, 2, 2)
        conv2_1 = layers.conv2d(pool1_stage1, 128, 3, 1,
                                activation_fn=None, scope='conv2_1')
        conv2_1 = tf.nn.relu(conv2_1)
        conv2_2 = layers.conv2d(
            conv2_1, 128, 3, 1, activation_fn=None, scope='conv2_2')
        conv2_2 = tf.nn.relu(conv2_2)
        pool2_stage1 = layers.max_pool2d(conv2_2, 2, 2)
        conv3_1 = layers.conv2d(pool2_stage1, 256, 3, 1,
                                activation_fn=None, scope='conv3_1')
        conv3_1 = tf.nn.relu(conv3_1)
        conv3_2 = layers.conv2d(
            conv3_1, 256, 3, 1, activation_fn=None, scope='conv3_2')
        conv3_2 = tf.nn.relu(conv3_2)
        conv3_3 = layers.conv2d(
            conv3_2, 256, 3, 1, activation_fn=None, scope='conv3_3')
        conv3_3 = tf.nn.relu(conv3_3)
        conv3_4 = layers.conv2d(
            conv3_3, 256, 3, 1, activation_fn=None, scope='conv3_4')
        conv3_4 = tf.nn.relu(conv3_4)
        pool3_stage1 = layers.max_pool2d(conv3_4, 2, 2)
        conv4_1 = layers.conv2d(pool3_stage1, 512, 3, 1,
                                activation_fn=None, scope='conv4_1')
        conv4_1 = tf.nn.relu(conv4_1)
        conv4_2 = layers.conv2d(
            conv4_1, 512, 3, 1, activation_fn=None, scope='conv4_2')
        conv4_2 = tf.nn.relu(conv4_2)
        conv4_3 = layers.conv2d(
            conv4_2, 512, 3, 1, activation_fn=None, scope='conv4_3')
        conv4_3 = tf.nn.relu(conv4_3)
        conv4_4 = layers.conv2d(
            conv4_3, 512, 3, 1, activation_fn=None, scope='conv4_4')
        conv4_4 = tf.nn.relu(conv4_4)
        conv5_1 = layers.conv2d(
            conv4_4, 512, 3, 1, activation_fn=None, scope='conv5_1')
        conv5_1 = tf.nn.relu(conv5_1)
        conv5_2_CPM = layers.conv2d(
            conv5_1, 128, 3, 1, activation_fn=None, scope='conv5_2_CPM')
        conv5_2_CPM = tf.nn.relu(conv5_2_CPM)
        conv6_1_CPM = layers.conv2d(
            conv5_2_CPM, 512, 1, 1, activation_fn=None, scope='conv6_1_CPM')
        conv6_1_CPM = tf.nn.relu(conv6_1_CPM)
        conv6_2_CPM = layers.conv2d(
            conv6_1_CPM, 1, 1, 1, activation_fn=None, scope='conv6_2_CPM')
        concat_stage2 = tf.concat([conv6_2_CPM, conv5_2_CPM], 3)
        Mconv1_stage2 = layers.conv2d(
            concat_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv1_stage2')
        Mconv1_stage2 = tf.nn.relu(Mconv1_stage2)
        Mconv2_stage2 = layers.conv2d(
            Mconv1_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv2_stage2')
        Mconv2_stage2 = tf.nn.relu(Mconv2_stage2)
        Mconv3_stage2 = layers.conv2d(
            Mconv2_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv3_stage2')
        Mconv3_stage2 = tf.nn.relu(Mconv3_stage2)
        Mconv4_stage2 = layers.conv2d(
            Mconv3_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv4_stage2')
        Mconv4_stage2 = tf.nn.relu(Mconv4_stage2)
        Mconv5_stage2 = layers.conv2d(
            Mconv4_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv5_stage2')
        Mconv5_stage2 = tf.nn.relu(Mconv5_stage2)
        Mconv6_stage2 = layers.conv2d(
            Mconv5_stage2, 128, 1, 1, activation_fn=None,
            scope='Mconv6_stage2')
        Mconv6_stage2 = tf.nn.relu(Mconv6_stage2)
        Mconv7_stage2 = layers.conv2d(
            Mconv6_stage2, 1, 1, 1, activation_fn=None, scope='Mconv7_stage2')
        concat_stage3 = tf.concat([Mconv7_stage2, conv5_2_CPM], 3)
        Mconv1_stage3 = layers.conv2d(
            concat_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv1_stage3')
        Mconv1_stage3 = tf.nn.relu(Mconv1_stage3)
        Mconv2_stage3 = layers.conv2d(
            Mconv1_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv2_stage3')
        Mconv2_stage3 = tf.nn.relu(Mconv2_stage3)
        Mconv3_stage3 = layers.conv2d(
            Mconv2_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv3_stage3')
        Mconv3_stage3 = tf.nn.relu(Mconv3_stage3)
        Mconv4_stage3 = layers.conv2d(
            Mconv3_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv4_stage3')
        Mconv4_stage3 = tf.nn.relu(Mconv4_stage3)
        Mconv5_stage3 = layers.conv2d(
            Mconv4_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv5_stage3')
        Mconv5_stage3 = tf.nn.relu(Mconv5_stage3)
        Mconv6_stage3 = layers.conv2d(
            Mconv5_stage3, 128, 1, 1, activation_fn=None,
            scope='Mconv6_stage3')
        Mconv6_stage3 = tf.nn.relu(Mconv6_stage3)
        Mconv7_stage3 = layers.conv2d(
            Mconv6_stage3, 1, 1, 1, activation_fn=None,
            scope='Mconv7_stage3')
        concat_stage4 = tf.concat([Mconv7_stage3, conv5_2_CPM], 3)
        Mconv1_stage4 = layers.conv2d(
            concat_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv1_stage4')
        Mconv1_stage4 = tf.nn.relu(Mconv1_stage4)
        Mconv2_stage4 = layers.conv2d(
            Mconv1_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv2_stage4')
        Mconv2_stage4 = tf.nn.relu(Mconv2_stage4)
        Mconv3_stage4 = layers.conv2d(
            Mconv2_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv3_stage4')
        Mconv3_stage4 = tf.nn.relu(Mconv3_stage4)
        Mconv4_stage4 = layers.conv2d(
            Mconv3_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv4_stage4')
        Mconv4_stage4 = tf.nn.relu(Mconv4_stage4)
        Mconv5_stage4 = layers.conv2d(
            Mconv4_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv5_stage4')
        Mconv5_stage4 = tf.nn.relu(Mconv5_stage4)
        Mconv6_stage4 = layers.conv2d(
            Mconv5_stage4, 128, 1, 1, activation_fn=None,
            scope='Mconv6_stage4')
        Mconv6_stage4 = tf.nn.relu(Mconv6_stage4)
        Mconv7_stage4 = layers.conv2d(
            Mconv6_stage4, 1, 1, 1, activation_fn=None, scope='Mconv7_stage4')
    return Mconv7_stage4


def _argmax_2d(tensor):
    """
    Compute argmax on the 2nd and 3d dimensions of the tensor.
    e.g. given an input tensor of size N x K x K x C, then it computes the (x,y) coordinates for
    each of the N images and C channels, corresponding to the max for that image and channel.
    :param tensor: image of size N x K x K x C
    :return: argmax in the format N x 2 x C (where C corresponds to NUM_JOINTS)
    """
    # get size
    shape = tensor.get_shape().as_list()[1]
    n_channels = tf.shape(tensor)[-1]

    # process each channel
    linearised_channel = tf.reshape(tensor, [-1, shape * shape, n_channels])
    best_channel = tf.argmax(linearised_channel, axis=1)

    idx_y = tf.expand_dims(tf.floordiv(best_channel, shape), axis=1)
    idx_x = tf.expand_dims(tf.mod(best_channel, shape), axis=1)
    argmax_channels = tf.concat([idx_x, idx_y], axis=1)
    return argmax_channels


def _process_stage(heat_maps, hm_size):
    """
    For each heat-map identify joint position and likelihood
    :param heat_maps: input heat-maps
    :param hm_size: size in which to return the coordinates
    :return: 2d joints (BATCH_SIZE x 14 x 2)
             likelihood for each joint (BATCH_SIZE x 14)
    """
    rescaled = tf.image.resize_images(heat_maps[:, :, :, :-1], [hm_size, hm_size])
    uncertainty = tf.reduce_max(tf.reduce_mean(rescaled, axis=1), axis=1)
    return _argmax_2d(rescaled), uncertainty


def inference_pose(image, center_map, hm_size, stage=6):
    with tf.variable_scope('PoseNet'):
        pool_center_lower = layers.avg_pool2d(center_map, 9, 8, padding='SAME')
        conv1_1 = layers.conv2d(
            image, 64, 3, 1, activation_fn=None, scope='conv1_1')
        conv1_1 = tf.nn.relu(conv1_1)
        conv1_2 = layers.conv2d(
            conv1_1, 64, 3, 1, activation_fn=None, scope='conv1_2')
        conv1_2 = tf.nn.relu(conv1_2)
        pool1_stage1 = layers.max_pool2d(conv1_2, 2, 2)
        conv2_1 = layers.conv2d(pool1_stage1, 128, 3, 1,
                                activation_fn=None, scope='conv2_1')
        conv2_1 = tf.nn.relu(conv2_1)
        conv2_2 = layers.conv2d(
            conv2_1, 128, 3, 1, activation_fn=None, scope='conv2_2')
        conv2_2 = tf.nn.relu(conv2_2)
        pool2_stage1 = layers.max_pool2d(conv2_2, 2, 2)
        conv3_1 = layers.conv2d(pool2_stage1, 256, 3, 1,
                                activation_fn=None, scope='conv3_1')
        conv3_1 = tf.nn.relu(conv3_1)
        conv3_2 = layers.conv2d(
            conv3_1, 256, 3, 1, activation_fn=None, scope='conv3_2')
        conv3_2 = tf.nn.relu(conv3_2)
        conv3_3 = layers.conv2d(
            conv3_2, 256, 3, 1, activation_fn=None, scope='conv3_3')
        conv3_3 = tf.nn.relu(conv3_3)
        conv3_4 = layers.conv2d(
            conv3_3, 256, 3, 1, activation_fn=None, scope='conv3_4')
        conv3_4 = tf.nn.relu(conv3_4)
        pool3_stage1 = layers.max_pool2d(conv3_4, 2, 2)
        conv4_1 = layers.conv2d(pool3_stage1, 512, 3, 1,
                                activation_fn=None, scope='conv4_1')
        conv4_1 = tf.nn.relu(conv4_1)
        conv4_2 = layers.conv2d(
            conv4_1, 512, 3, 1, activation_fn=None, scope='conv4_2')
        conv4_2 = tf.nn.relu(conv4_2)
        conv4_3_CPM = layers.conv2d(
            conv4_2, 256, 3, 1, activation_fn=None, scope='conv4_3_CPM')
        conv4_3_CPM = tf.nn.relu(conv4_3_CPM)
        conv4_4_CPM = layers.conv2d(
            conv4_3_CPM, 256, 3, 1, activation_fn=None, scope='conv4_4_CPM')
        conv4_4_CPM = tf.nn.relu(conv4_4_CPM)
        conv4_5_CPM = layers.conv2d(
            conv4_4_CPM, 256, 3, 1, activation_fn=None, scope='conv4_5_CPM')
        conv4_5_CPM = tf.nn.relu(conv4_5_CPM)
        conv4_6_CPM = layers.conv2d(
            conv4_5_CPM, 256, 3, 1, activation_fn=None, scope='conv4_6_CPM')
        conv4_6_CPM = tf.nn.relu(conv4_6_CPM)
        conv4_7_CPM = layers.conv2d(
            conv4_6_CPM, 128, 3, 1, activation_fn=None, scope='conv4_7_CPM')
        conv4_7_CPM = tf.nn.relu(conv4_7_CPM)
        conv5_1_CPM = layers.conv2d(
            conv4_7_CPM, 512, 1, 1, activation_fn=None, scope='conv5_1_CPM')
        conv5_1_CPM = tf.nn.relu(conv5_1_CPM)
        conv5_2_CPM = layers.conv2d(
            conv5_1_CPM, 15, 1, 1, activation_fn=None, scope='conv5_2_CPM')
        concat_stage2 = tf.concat(
            [conv5_2_CPM, conv4_7_CPM, pool_center_lower], 3)
        Mconv1_stage2 = layers.conv2d(
            concat_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv1_stage2')
        Mconv1_stage2 = tf.nn.relu(Mconv1_stage2)
        Mconv2_stage2 = layers.conv2d(
            Mconv1_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv2_stage2')
        Mconv2_stage2 = tf.nn.relu(Mconv2_stage2)
        Mconv3_stage2 = layers.conv2d(
            Mconv2_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv3_stage2')
        Mconv3_stage2 = tf.nn.relu(Mconv3_stage2)
        Mconv4_stage2 = layers.conv2d(
            Mconv3_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv4_stage2')
        Mconv4_stage2 = tf.nn.relu(Mconv4_stage2)
        Mconv5_stage2 = layers.conv2d(
            Mconv4_stage2, 128, 7, 1, activation_fn=None,
            scope='Mconv5_stage2')
        Mconv5_stage2 = tf.nn.relu(Mconv5_stage2)
        Mconv6_stage2 = layers.conv2d(
            Mconv5_stage2, 128, 1, 1, activation_fn=None,
            scope='Mconv6_stage2')
        Mconv6_stage2 = tf.nn.relu(Mconv6_stage2)
        Mconv7_stage2 = layers.conv2d(
            Mconv6_stage2, 15, 1, 1, activation_fn=None, scope='Mconv7_stage2')
        if stage == 2:
            return _process_stage(Mconv7_stage2, hm_size)

        concat_stage3 = tf.concat(
            [Mconv7_stage2, conv4_7_CPM, pool_center_lower], 3)
        Mconv1_stage3 = layers.conv2d(
            concat_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv1_stage3')
        Mconv1_stage3 = tf.nn.relu(Mconv1_stage3)
        Mconv2_stage3 = layers.conv2d(
            Mconv1_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv2_stage3')
        Mconv2_stage3 = tf.nn.relu(Mconv2_stage3)
        Mconv3_stage3 = layers.conv2d(
            Mconv2_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv3_stage3')
        Mconv3_stage3 = tf.nn.relu(Mconv3_stage3)
        Mconv4_stage3 = layers.conv2d(
            Mconv3_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv4_stage3')
        Mconv4_stage3 = tf.nn.relu(Mconv4_stage3)
        Mconv5_stage3 = layers.conv2d(
            Mconv4_stage3, 128, 7, 1, activation_fn=None,
            scope='Mconv5_stage3')
        Mconv5_stage3 = tf.nn.relu(Mconv5_stage3)
        Mconv6_stage3 = layers.conv2d(
            Mconv5_stage3, 128, 1, 1, activation_fn=None,
            scope='Mconv6_stage3')
        Mconv6_stage3 = tf.nn.relu(Mconv6_stage3)
        Mconv7_stage3 = layers.conv2d(
            Mconv6_stage3, 15, 1, 1, activation_fn=None, scope='Mconv7_stage3')
        if stage == 3:
            return _process_stage(Mconv7_stage3, hm_size)

        concat_stage4 = tf.concat(
            [Mconv7_stage3, conv4_7_CPM, pool_center_lower], 3)
        Mconv1_stage4 = layers.conv2d(
            concat_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv1_stage4')
        Mconv1_stage4 = tf.nn.relu(Mconv1_stage4)
        Mconv2_stage4 = layers.conv2d(
            Mconv1_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv2_stage4')
        Mconv2_stage4 = tf.nn.relu(Mconv2_stage4)
        Mconv3_stage4 = layers.conv2d(
            Mconv2_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv3_stage4')
        Mconv3_stage4 = tf.nn.relu(Mconv3_stage4)
        Mconv4_stage4 = layers.conv2d(
            Mconv3_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv4_stage4')
        Mconv4_stage4 = tf.nn.relu(Mconv4_stage4)
        Mconv5_stage4 = layers.conv2d(
            Mconv4_stage4, 128, 7, 1, activation_fn=None,
            scope='Mconv5_stage4')
        Mconv5_stage4 = tf.nn.relu(Mconv5_stage4)
        Mconv6_stage4 = layers.conv2d(
            Mconv5_stage4, 128, 1, 1, activation_fn=None,
            scope='Mconv6_stage4')
        Mconv6_stage4 = tf.nn.relu(Mconv6_stage4)
        Mconv7_stage4 = layers.conv2d(
            Mconv6_stage4, 15, 1, 1, activation_fn=None, scope='Mconv7_stage4')
        if stage == 4:
            return _process_stage(Mconv7_stage4, hm_size)

        concat_stage5 = tf.concat(
            [Mconv7_stage4, conv4_7_CPM, pool_center_lower], 3)
        Mconv1_stage5 = layers.conv2d(
            concat_stage5, 128, 7, 1, activation_fn=None,
            scope='Mconv1_stage5')
        Mconv1_stage5 = tf.nn.relu(Mconv1_stage5)
        Mconv2_stage5 = layers.conv2d(
            Mconv1_stage5, 128, 7, 1, activation_fn=None,
            scope='Mconv2_stage5')
        Mconv2_stage5 = tf.nn.relu(Mconv2_stage5)
        Mconv3_stage5 = layers.conv2d(
            Mconv2_stage5, 128, 7, 1, activation_fn=None,
            scope='Mconv3_stage5')
        Mconv3_stage5 = tf.nn.relu(Mconv3_stage5)
        Mconv4_stage5 = layers.conv2d(
            Mconv3_stage5, 128, 7, 1, activation_fn=None,
            scope='Mconv4_stage5')
        Mconv4_stage5 = tf.nn.relu(Mconv4_stage5)
        Mconv5_stage5 = layers.conv2d(
            Mconv4_stage5, 128, 7, 1, activation_fn=None,
            scope='Mconv5_stage5')
        Mconv5_stage5 = tf.nn.relu(Mconv5_stage5)
        Mconv6_stage5 = layers.conv2d(
            Mconv5_stage5, 128, 1, 1, activation_fn=None,
            scope='Mconv6_stage5')
        Mconv6_stage5 = tf.nn.relu(Mconv6_stage5)
        Mconv7_stage5 = layers.conv2d(
            Mconv6_stage5, 15, 1, 1, activation_fn=None, scope='Mconv7_stage5')
        if stage == 5:
            return _process_stage(Mconv7_stage5, hm_size)

        concat_stage6 = tf.concat(
            [Mconv7_stage5, conv4_7_CPM, pool_center_lower], 3)
        Mconv1_stage6 = layers.conv2d(
            concat_stage6, 128, 7, 1, activation_fn=None,
            scope='Mconv1_stage6')
        Mconv1_stage6 = tf.nn.relu(Mconv1_stage6)
        Mconv2_stage6 = layers.conv2d(
            Mconv1_stage6, 128, 7, 1, activation_fn=None,
            scope='Mconv2_stage6')
        Mconv2_stage6 = tf.nn.relu(Mconv2_stage6)
        Mconv3_stage6 = layers.conv2d(
            Mconv2_stage6, 128, 7, 1, activation_fn=None,
            scope='Mconv3_stage6')
        Mconv3_stage6 = tf.nn.relu(Mconv3_stage6)
        Mconv4_stage6 = layers.conv2d(
            Mconv3_stage6, 128, 7, 1, activation_fn=None,
            scope='Mconv4_stage6')
        Mconv4_stage6 = tf.nn.relu(Mconv4_stage6)
        Mconv5_stage6 = layers.conv2d(
            Mconv4_stage6, 128, 7, 1, activation_fn=None,
            scope='Mconv5_stage6')
        Mconv5_stage6 = tf.nn.relu(Mconv5_stage6)
        Mconv6_stage6 = layers.conv2d(
            Mconv5_stage6, 128, 1, 1, activation_fn=None,
            scope='Mconv6_stage6')
        Mconv6_stage6 = tf.nn.relu(Mconv6_stage6)
        Mconv7_stage6 = layers.conv2d(
            Mconv6_stage6, 15, 1, 1, activation_fn=None,
            scope='Mconv7_stage6')
        return _process_stage(Mconv7_stage6, hm_size)



draw

In [ ]:
import cv2
import numpy as np
#from .config import JOINT_DRAW_SIZE
#from .config import LIMB_DRAW_SIZE
#from .config import NORMALISATION_COEFFICIENT
import matplotlib.pyplot as plt
import math

__all__ = [
    'draw_limbs',
    'plot_pose'
]


def draw_limbs(image, pose_2d, visible):
    """Draw the 2D pose without the occluded/not visible joints."""

    _COLORS = [
        [0, 0, 255], [0, 170, 255], [0, 255, 170], [0, 255, 0],
        [170, 255, 0], [255, 170, 0], [255, 0, 0], [255, 0, 170],
        [170, 0, 255]
    ]
    _LIMBS = np.array([0, 1, 2, 3, 3, 4, 5, 6, 6, 7, 8, 9,
                       9, 10, 11, 12, 12, 13]).reshape((-1, 2))

    _NORMALISATION_FACTOR = int(math.floor(math.sqrt(image.shape[0] * image.shape[1] / NORMALISATION_COEFFICIENT)))

    for oid in range(pose_2d.shape[0]):
        for lid, (p0, p1) in enumerate(_LIMBS):
            if not (visible[oid][p0] and visible[oid][p1]):
                continue
            y0, x0 = pose_2d[oid][p0]
            y1, x1 = pose_2d[oid][p1]
            cv2.circle(image, (x0, y0), JOINT_DRAW_SIZE *_NORMALISATION_FACTOR , _COLORS[lid], -1)
            cv2.circle(image, (x1, y1), JOINT_DRAW_SIZE*_NORMALISATION_FACTOR , _COLORS[lid], -1)
            cv2.line(image, (x0, y0), (x1, y1),
                     _COLORS[lid], LIMB_DRAW_SIZE*_NORMALISATION_FACTOR , 16)


def plot_pose(pose):
    """Plot the 3D pose showing the joint connections."""
    import mpl_toolkits.mplot3d.axes3d as p3

    _CONNECTION = [
        [0, 1], [1, 2], [2, 3], [0, 4], [4, 5], [5, 6], [0, 7], [7, 8],
        [8, 9], [9, 10], [8, 11], [11, 12], [12, 13], [8, 14], [14, 15],
        [15, 16]]

    def joint_color(j):
        """
        TODO: 'j' shadows name 'j' from outer scope
        """

        colors = [(0, 0, 0), (255, 0, 255), (0, 0, 255),
                  (0, 255, 255), (255, 0, 0), (0, 255, 0)]
        _c = 0
        if j in range(1, 4):
            _c = 1
        if j in range(4, 7):
            _c = 2
        if j in range(9, 11):
            _c = 3
        if j in range(11, 14):
            _c = 4
        if j in range(14, 17):
            _c = 5
        return colors[_c]

    assert (pose.ndim == 2)
    assert (pose.shape[0] == 3)
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    for c in _CONNECTION:
        col = '#%02x%02x%02x' % joint_color(c[0])
        ax.plot([pose[0, c[0]], pose[0, c[1]]],
                [pose[1, c[0]], pose[1, c[1]]],
                [pose[2, c[0]], pose[2, c[1]]], c=col)
    for j in range(pose.shape[1]):
        col = '#%02x%02x%02x' % joint_color(j)
        ax.scatter(pose[0, j], pose[1, j], pose[2, j],
                   c=col, marker='o', edgecolor=col)
    smallest = pose.min()
    largest = pose.max()
    ax.set_xlim3d(smallest, largest)
    ax.set_ylim3d(smallest, largest)
    ax.set_zlim3d(smallest, largest)

    return fig




upright_fast

In [ ]:
import numpy as np
import scipy

__all__ = [
    'upgrade_r',
    'update_cam',
    'estimate_a_and_r_with_res',
    'estimate_a_and_r_with_res_weights',
    'pick_e'
]


def upgrade_r(r):
    """Upgrades complex parameterisation of planar rotation to tensor containing
    per frame 3x3 rotation matrices"""
    newr = np.zeros((3, 3))
    newr[:2, 0] = r
    newr[2, 2] = 1
    newr[1::-1, 1] = r
    newr[0, 1] *= -1
    return newr


def update_cam(cam):
    new_cam = cam[[0, 2, 1]].copy()
    new_cam = new_cam[:, [0, 2, 1]]
    return new_cam


def estimate_a_and_r_with_res(
        w, e, s0, camera_r, Lambda, check, a, weights, res, proj_e,
        residue, Ps, depth_reg, scale_prior):
    """
    TODO: Missing the following parameters in docstring:
        - w, e, s0, camera_r, Lambda, check, a, res, proj_e, depth_reg,
          scale_prior

    TODO: The following parameters are not used:
        - s0, weights

    So local optima are a problem in general.
    However:

        1. This problem is convex in a but not in r, and

        2. each frame can be solved independently.

    So for each frame, we can do a grid search in r and take the globally
    optimal solution.

    In practice, we just brute force over 100 different estimates of r, and
    take the best pair (r,a*(r)) where a*(r) is the optimal minimiser of a
    given r.

    Arguments:

        w is a 3d measurement matrix of form frames*2*points

        e is a 3d set of basis vectors of from basis*3*points

        s0 is the 3d rest shape of form 3*points

        Lambda are the regularisor coefficients on the coefficients of the
        weights typically generated using PPCA

        interval is how far round the circle we should check for break points
        we check every interval*2*pi radians

    Returns:

        a (basis coefficients) and r (representation of rotations as a complex
        number)
    """
    frames = w.shape[0]
    points = w.shape[2]
    basis = e.shape[0]
    r = np.empty(2)
    Ps_reshape = Ps.reshape(2 * points)
    w_reshape = w.reshape((frames, points * 2))

    for i in range(check.size):
        c = check[i]
        r[0] = np.cos(c)
        r[1] = np.sin(c)
        grot = camera_r.dot(upgrade_r(r))
        rot = grot[:2]
        res[:, :points * 2] = w_reshape
        res[:, :points * 2] -= Ps_reshape
        proj_e[:, :2 * points] = rot.dot(e).transpose(1, 0, 2).reshape(
            e.shape[0], 2 * points)

        if Lambda.size != 0:
            proj_e[:, 2 * points:2 * points + basis] = np.diag(Lambda[:Lambda.shape[0] - 1])
            res[:, 2 * points:].fill(0)
            res[:, :points * 2] *= Lambda[Lambda.shape[0] - 1]
            proj_e[:, :points * 2] *= Lambda[Lambda.shape[0] - 1]
            # depth regularizer not used
            proj_e[:, 2 * points + basis:] = ((Lambda[Lambda.shape[0] - 1] *
                                               depth_reg) * grot[2]).dot(e)
            # we let the person change scale
            res[:, 2 * points] = scale_prior

        """
        TODO: PLEASE REVIEW THE FOLLOWING CODE....
        overwrite_a and overwrite_b ARE UNEXPECTED ARGUMENTS OF
        scipy.linalg.lstsq
        """
        a[i], residue[i], _, _ = scipy.linalg.lstsq(
            proj_e.T, res.T, overwrite_a=True, overwrite_b=True)

    # find and return best coresponding solution
    best = np.argmin(residue, 0)
    assert (best.shape[0] == frames)
    theta = check[best]
    index = (best, np.arange(frames))
    aa = a.transpose(0, 2, 1)[index]
    retres = residue[index]
    r = np.empty((2, frames))
    r[0] = np.sin(theta)
    r[1] = np.cos(theta)
    return aa, r, retres


def estimate_a_and_r_with_res_weights(
        w, e, s0, camera_r, Lambda, check, a, weights, res, proj_e,
        residue, Ps, depth_reg, scale_prior):
    """
    TODO: Missing the following parameters in docstring:
     - w, e, s0, camera)r, Lambda, check, a, res, proj_e, residue,
     Ps, depth_reg, scale_prior

    So local optima are a problem in general.
    However:

        1. This problem is convex in a but not in r, and

        2. each frame can be solved independently.

    So for each frame, we can do a grid search in r and take the globally
    optimal solution.

    In practice, we just brute force over 100 different estimates of r, and
    take
    the best pair (r,a*(r)) where a*(r) is the optimal minimiser of a given r.

    Arguments:

        w is a 3d measurement matrix of form frames*2*points

        e is a 3d set of basis vectors of from basis*3*points

        s0 is the 3d rest shape of form 3*points

        Lambda are the regularisor coefficients on the coefficients of the
        weights
        typically generated using PPCA

        interval is how far round the circle we should check for break points
        we check every interval*2*pi radians

    Returns:

        a (basis coefficients) and r (representation of rotations as a complex
        number)
    """
    frames = w.shape[0]
    points = w.shape[2]
    basis = e.shape[0]
    r = np.empty(2)
    Ps_reshape = Ps.reshape(2 * points)
    w_reshape = w.reshape((frames, points * 2))
    p_copy = np.empty_like(proj_e)

    for i in range(check.size):
        c = check[i]
        r[0] = np.sin(c)
        r[1] = np.cos(c)
        grot = camera_r.dot(upgrade_r(r).T)
        rot = grot[:2]
        rot.dot(s0, Ps)  # TODO: remove?
        res[:, :points * 2] = w_reshape
        res[:, :points * 2] -= Ps_reshape
        proj_e[:, :2 * points] = rot.dot(e).transpose(1, 0, 2).reshape(
            e.shape[0], 2 * points)

        if Lambda.size != 0:
            proj_e[:, 2 * points:2 * points + basis] = np.diag(Lambda[:Lambda.shape[0] - 1])
            res[:, 2 * points:].fill(0)
            res[:, :points * 2] *= Lambda[Lambda.shape[0] - 1]
            proj_e[:, :points * 2] *= Lambda[Lambda.shape[0] - 1]
            proj_e[:, 2 * points + basis:] = ((Lambda[Lambda.shape[0] - 1] *
                                               depth_reg) * grot[2]).dot(e)
            res[:, 2 * points] = scale_prior
        if weights.size != 0:
            res[:, :points * 2] *= weights
        for j in range(frames):
            p_copy[:] = proj_e
            p_copy[:, :points * 2] *= weights[j]
            a[i, :, j], comp_residual, _, _ = np.linalg.lstsq(
                p_copy.T, res[j].T)
            if not comp_residual:
                # equations are over-determined
                residue[i, j] = 1e-5
            else:
                residue[i, j] = comp_residual
    # find and return best coresponding solution
    best = np.argmin(residue, 0)
    index = (best, np.arange(frames))
    theta = check[best]
    aa = a.transpose(0, 2, 1)[index]
    retres = residue[index]
    r = np.empty((2, frames))
    r[0] = np.sin(theta)
    r[1] = np.cos(theta)
    return aa, r, retres


def pick_e(w, e, s0, camera_r=None, Lambda=None,
           weights=None, scale_prior=-0.0014, interval=0.01, depth_reg=0.0325):
    """Brute force over charts from the manifold to find the best one.
        Returns best chart index and its a and r coefficients
        Returns assignment, and a and r coefficents"""

    camera_r = np.asarray([[1, 0, 0], [0, 0, -1], [0, 1, 0]]
                          ) if camera_r is None else camera_r
    Lambda = np.ones((0, 0)) if Lambda is None else Lambda
    weights = np.ones((0, 0, 0)) if weights is None else weights

    charts = e.shape[0]
    frames = w.shape[0]
    basis = e.shape[1]
    points = e.shape[3]
    assert (s0.shape[0] == charts)
    r = np.empty((charts, 2, frames))
    a = np.empty((charts, frames, e.shape[1]))
    score = np.empty((charts, frames))
    check = np.arange(0, 1, interval) * 2 * np.pi
    cache_a = np.empty((check.size, basis, frames))
    residue = np.empty((check.size, frames))

    if Lambda.size != 0:
        res = np.zeros((frames, points * 2 + basis + points))
        proj_e = np.zeros((basis, 2 * points + basis + points))
    else:
        res = np.empty((frames, points * 2))
        proj_e = np.empty((basis, 2 * points))
    Ps = np.empty((2, points))

    if weights.size == 0:
        for i in range(charts):
            if Lambda.size != 0:
                a[i], r[i], score[i] = estimate_a_and_r_with_res(
                    w, e[i], s0[i], camera_r,
                    Lambda[i], check, cache_a, weights,
                    res, proj_e, residue, Ps,
                    depth_reg, scale_prior)
            else:
                a[i], r[i], score[i] = estimate_a_and_r_with_res(
                    w, e[i], s0[i], camera_r, Lambda,
                    check, cache_a, weights,
                    res, proj_e, residue, Ps,
                    depth_reg, scale_prior)
    else:
        w2 = weights.reshape(weights.shape[0], -1)
        for i in range(charts):
            if Lambda.size != 0:
                a[i], r[i], score[i] = estimate_a_and_r_with_res_weights(
                    w, e[i], s0[i], camera_r,
                    Lambda[i], check, cache_a, w2,
                    res, proj_e, residue, Ps,
                    depth_reg, scale_prior)
            else:
                a[i], r[i], score[i] = estimate_a_and_r_with_res_weights(
                    w, e[i], s0[i], camera_r, Lambda,
                    check, cache_a, w2,
                    res, proj_e, residue, Ps,
                    depth_reg, scale_prior)

    remaining_dims = 3 * w.shape[2] - e.shape[1]
    assert (np.all(score > 0))
    assert (remaining_dims >= 0)
    # Zero problems in log space due to un-regularised first co-efficient
    l = Lambda.copy()
    l[Lambda == 0] = 1
    llambda = -np.log(l)
    score /= 2
    return score, a, r


process

In [ ]:
from __future__ import division

import os
import json
import numpy as np
#from lifting.utils import config
import cv2
import skimage.io
import skimage.transform
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
from itertools import compress
from scipy.stats import multivariate_normal

__all__ = [
    'detect_objects_heatmap',
    'detect_objects_heatmap',
    'gaussian_kernel',
    'gaussian_heatmap',
    'prepare_input_posenet',
    'detect_parts_heatmaps',
    'detect_parts_from_likelihoods',
    'import_json',
    'generate_labels',
    'generate_center_map',
    'rescale',
    'crop_image'
]


def detect_objects_heatmap(heatmap):
    data = 256 * heatmap
    data_max = filters.maximum_filter(data, 3)
    maxima = (data == data_max)
    data_min = filters.minimum_filter(data, 3)
    diff = ((data_max - data_min) > 0.3)
    maxima[diff == 0] = 0
    labeled, num_objects = ndimage.label(maxima)
    slices = ndimage.find_objects(labeled)
    objects = np.zeros((num_objects, 2), dtype=np.int32)
    pidx = 0
    for (dy, dx) in slices:
        pos = [(dy.start + dy.stop - 1) // 2, (dx.start + dx.stop - 1) // 2]
        if heatmap[pos[0], pos[1]] > CENTER_TR:
            objects[pidx, :] = pos
            pidx += 1
    return objects[:pidx]


def gaussian_kernel(h, w, sigma_h, sigma_w):
    yx = np.mgrid[-h // 2:h // 2, -w // 2:w // 2] ** 2
    return np.exp(-yx[0, :, :] / sigma_h ** 2 - yx[1, :, :] / sigma_w ** 2)


def gaussian_heatmap(h, w, pos_x, pos_y, sigma_h=1, sigma_w=1, init=None):
    """
    Compute the heat-map of size (w x h) with a gaussian distribution fit in
    position (pos_x, pos_y) and a convariance matix defined by the related
    sigma values.
    The resulting heat-map can be summed to a given heat-map init.
    """
    init = init if init is not None else []

    cov_matrix = np.eye(2) * ([sigma_h**2, sigma_w**2])

    x, y = np.mgrid[0:h, 0:w]
    pos = np.dstack((x, y))
    rv = multivariate_normal([pos_x, pos_y], cov_matrix)

    tmp = rv.pdf(pos)
    hmap = np.multiply(
        tmp, np.sqrt(np.power(2 * np.pi, 2) * np.linalg.det(cov_matrix))
    )
    idx = np.where(hmap.flatten() <= np.exp(-4.6052))
    hmap.flatten()[idx] = 0

    if np.size(init) == 0:
        return hmap

    assert (np.shape(init) == hmap.shape)
    hmap += init
    idx = np.where(hmap.flatten() > 1)
    hmap.flatten()[idx] = 1
    return hmap


def prepare_input_posenet(image, objects, size_person, size,
                          batch_size, sigma=25, border=400):
    result = np.zeros((batch_size, size[0], size[1], 4))
    padded_image = np.zeros(
        (1, size_person[0] + border, size_person[1] + border, 4))
    padded_image[0, border // 2:-border // 2,
                 border // 2:-border // 2, :3] = image
    if objects.shape[0] > batch_size:
        objects = objects[:batch_size]
    for oid, (yc, xc) in enumerate(objects):
        dh, dw = size[0] // 2, size[1] // 2
        y0, x0, y1, x1 = np.array(
            [yc - dh, xc - dw, yc + dh, xc + dw]) + border // 2
        result[oid, :, :, :4] = padded_image[:, y0:y1, x0:x1, :]
        result[oid, :, :, 3] = gaussian_kernel(size[0], size[1], sigma, sigma)
    return np.split(result, [3], 3)


def detect_parts_heatmaps(heatmaps, centers, size, num_parts=14):
    """
    Given heat-maps find the position of each joint by means of n argmax
    function
    """
    parts = np.zeros((len(centers), num_parts, 2), dtype=np.int32)
    visible = np.zeros((len(centers), num_parts), dtype=bool)
    for oid, (yc, xc) in enumerate(centers):
        part_hmap = skimage.transform.resize(
            np.clip(heatmaps[oid], -1, 1), size)
        for pid in range(num_parts):
            y, x = np.unravel_index(np.argmax(part_hmap[:, :, pid]), size)
            parts[oid, pid] = y + yc - size[0] // 2, x + xc - size[1] // 2
            visible[oid, pid] = np.mean(
                part_hmap[:, :, pid]) > VISIBLE_PART
    return parts, visible


def detect_parts_from_likelihoods(poses, centers, likelihoods, num_parts=14):
    """
    Given heat-maps find the position of each joint by means of n argmax
    function
    """
    if len(centers) > BATCH_SIZE:
        centers = centers[:BATCH_SIZE]
    parts = np.zeros((len(centers), num_parts, 2), dtype=np.int32)
    visible = np.zeros((len(centers), num_parts), dtype=bool)
    for oid, (yc, xc) in enumerate(centers):
        for pid in range(num_parts):
            x, y = poses[oid, :, pid]
            parts[oid, pid] = y + yc - INPUT_SIZE // 2, x + xc - INPUT_SIZE // 2
            visible[oid, pid] = likelihoods[oid, pid] > VISIBLE_PART
    return parts, visible


def import_json(path='json/MPI_annotations.json', order='json/MPI_order.npy'):
    """Get the json file containing the dataset.
    We want the data to be shuffled, however the training has to be repeatable.
    This means that once shuffled the order has to me mantained."""
    with open(path) as data_file:
        data_this = json.load(data_file)
        data_this = np.array(data_this['root'])
    num_samples = len(data_this)

    if os.path.exists(order):
        idx = np.load(order)
    else:
        idx = np.random.permutation(num_samples).tolist()
        np.save(order, idx)

    is_not_validation = [not data_this[i]['isValidation']
                         for i in range(num_samples)]
    keep_data_idx = list(compress(idx, is_not_validation))

    data = data_this[keep_data_idx]
    return data, len(keep_data_idx)


def generate_labels(image_shape, joint_positions, num_other_people,
                    joints_other_people, offset):
    """
    Given as input a set of joint positions and the size of the input image
    it generates
    a set of heat-maps of the same size. It generates both heat-maps used as
    labels for the first stage (label_1st_lower) and for all the other stages
    (label_lower).
    """
    _FILTER_JOINTS = np.array([9, 8, 12, 11, 10, 13, 14, 15, 2, 1, 0, 3, 4, 5])

    img_height, img_width, _ = image_shape
    heat_maps_single_p = np.zeros(
        (NUM_OUTPUT, INPUT_SIZE, INPUT_SIZE))
    heat_maps_other_p = np.zeros(
        (NUM_OUTPUT, INPUT_SIZE, INPUT_SIZE))

    # generate first set of heat-maps
    for i in range(NUM_JOINTS):
        # the set of joints can be different fromt the one in the json file
        curr_joint = joint_positions[_FILTER_JOINTS[i]]
        skip = (curr_joint[0] < 0 or curr_joint[1] < 0 or
                curr_joint[0] >= img_width or curr_joint[1] >= img_height)
        if not skip:
            heat_maps_single_p[i] = gaussian_heatmap(
                INPUT_SIZE, INPUT_SIZE,
                curr_joint[
                    1] - offset[1], curr_joint[0] - offset[0],
                SIGMA, SIGMA)

            heat_maps_other_p[i] = gaussian_heatmap(
                INPUT_SIZE, INPUT_SIZE,
                curr_joint[
                    1] - offset[1], curr_joint[0] - offset[0],
                SIGMA, SIGMA)

    heat_maps_single_p[-1] = np.maximum(
        1 - np.max(heat_maps_single_p[:-1], axis=0),
        np.zeros((INPUT_SIZE, INPUT_SIZE)))
    heat_maps_single_p = np.transpose(heat_maps_single_p, (1, 2, 0))

    # generate second set of heat-maps for other people in the image
    for p in range(int(num_other_people)):
        for i in range(NUM_JOINTS):
            # the set of joints can be different fromt the one in the json file
            try:
                if num_other_people == 1:
                    curr_joint = joints_other_people[_FILTER_JOINTS[i]]
                else:
                    curr_joint = joints_other_people[p][_FILTER_JOINTS[i]]
                skip = (
                    curr_joint[0] < 0 or curr_joint[1] < 0 or
                    curr_joint[0] >= img_width or curr_joint[1] >= img_height)
            except IndexError:
                skip = True
            if not skip:
                heat_maps_other_p[i] = gaussian_heatmap(
                    INPUT_SIZE, INPUT_SIZE,
                    curr_joint[1] - offset[1], curr_joint[0] - offset[0],
                    SIGMA, SIGMA, init=heat_maps_other_p[i])

    heat_maps_other_p[-1] = np.maximum(
        1 - np.max(heat_maps_other_p[:-1], axis=0),
        np.zeros((INPUT_SIZE, INPUT_SIZE)))

    heat_maps_other_p = np.transpose(heat_maps_other_p, (1, 2, 0))

    # rescaling heat-maps accoring to the right shape
    labels_single = rescale(heat_maps_single_p, OUTPUT_SIZE)
    labels_people = rescale(heat_maps_other_p, OUTPUT_SIZE)
    return labels_people, labels_single


def generate_center_map(center_pos, img_shape):
    """
    Given the position of the person and the size of the input image it
    generates
    a heat-map where a gaissian distribution is fit in the position of the
    person in the image.
    """
    img_height = img_shape
    img_width = img_shape
    center_map = gaussian_heatmap(
        img_height, img_width, center_pos[1], center_pos[0],
        SIGMA_CENTER, SIGMA_CENTER)
    return center_map


def rescale(data, new_size):
    """Rescale data to a fixed dimension, regardless the number of channels.
    Data has to be in the format (h,w,c)."""
    if data.ndim > 2:
        assert data.shape[2] < data.shape[0]
        assert data.shape[2] < data.shape[1]
    resized_data = cv2.resize(
        data, (new_size, new_size), interpolation=cv2.INTER_CUBIC)
    return resized_data


def crop_image(image, obj_pose):
    """
    Crop the image in order to have the person at the center and the final
    image size
    is the same as the expected CNN input size.
    Returns the cropped image and the offset that is used to update the joint
    positions.
    """
    offset_left = int(obj_pose[0] - INPUT_SIZE // 2)
    offset_up = int(obj_pose[1] - INPUT_SIZE // 2)
    # just for checking that it's inside the image
    offset_right = int(image.shape[1] - obj_pose[0] - INPUT_SIZE // 2)
    offset_bottom = int(image.shape[0] - obj_pose[1] - INPUT_SIZE // 2)

    pad_left, pad_right, pad_up, pad_bottom = 0, 0, 0, 0
    if offset_left < 0:
        pad_left = -offset_left
    if offset_right < 0:
        pad_right = -offset_right
    if offset_up < 0:
        pad_up = -offset_up
    if offset_bottom < 0:
        pad_bottom = -offset_bottom
    padded_image = np.lib.pad(
        image, ((pad_up, pad_bottom), (pad_left, pad_right), (0, 0)),
        'constant', constant_values=((0, 0), (0, 0), (0, 0)))

    cropped_image = padded_image[
        offset_up + pad_up: offset_up + pad_up + INPUT_SIZE,
        offset_left + pad_left: offset_left + pad_left + INPUT_SIZE]

    return cropped_image, np.array([offset_left, offset_up])


prob_model

In [ ]:
import os
import scipy.io as sio
import numpy as np
#from lifting.utils.upright_fast import pick_e
#from lifting.utils import config

__all__ = ['Prob3dPose']


class Prob3dPose:

    def __init__(self, prob_model_path):
        model_param = sio.loadmat(prob_model_path)
        self.mu = np.reshape(
            model_param['mu'], (model_param['mu'].shape[0], 3, -1))
        self.e = np.reshape(model_param['e'], (model_param['e'].shape[
                            0], model_param['e'].shape[1], 3, -1))
        self.sigma = model_param['sigma']
        self.cam = np.array(
            [[1.0, 0.0, 0.0], [0.0, 0.0, -1.0], [0.0, 1.0, 0.0]])

    @staticmethod
    def cost3d(model, gt):
        """3d error in mm"""
        out = np.sqrt(((gt - model) ** 2).sum(1)).mean(-1)
        return out

    @staticmethod
    def renorm_gt(gt):
        """Compel gt data to have mean joint length of one"""
        _POSE_TREE = np.asarray([
            [0, 1], [1, 2], [2, 3], [0, 4], [4, 5], [5, 6], [0, 7], [7, 8],
            [8, 9], [9, 10], [8, 11], [11, 12], [12, 13], [8, 14], [14, 15],
            [15, 16]]).T
        scale = np.sqrt(((gt[:, :, _POSE_TREE[0]] -
                          gt[:, :, _POSE_TREE[1]]) ** 2).sum(2).sum(1))
        return gt / scale[:, np.newaxis, np.newaxis]

    @staticmethod
    def build_model(a, e, s0):
        """Build 3D model"""
        assert (s0.shape[1] == 3)
        assert (e.shape[2] == 3)
        assert (a.shape[1] == e.shape[1])
        out = np.einsum('...i,...ijk', a, e)
        out += s0
        return out

    @staticmethod
    def build_and_rot_model(a, e, s0, r):
        """
        Build model and rotate according to the identified rotation matrix
        """
        from numpy.core.umath_tests import matrix_multiply

        r2 = Prob3dPose.upgrade_r(r.T).transpose((0, 2, 1))
        mod = Prob3dPose.build_model(a, e, s0)
        mod = matrix_multiply(r2, mod)
        return mod

    @staticmethod
    def upgrade_r(r):
        """
        Upgrades complex parameterisation of planar rotation to tensor
        containing per frame 3x3 rotation matrices
        """
        assert (r.ndim == 2)
        # Technically optional assert, but if this fails data is probably
        # transposed
        assert (r.shape[1] == 2)
        assert (np.all(np.isfinite(r)))
        norm = np.sqrt((r[:, :2] ** 2).sum(1))
        assert (np.all(norm > 0))
        r /= norm[:, np.newaxis]
        assert (np.all(np.isfinite(r)))
        newr = np.zeros((r.shape[0], 3, 3))
        newr[:, :2, 0] = r[:, :2]
        newr[:, 2, 2] = 1
        newr[:, 1::-1, 1] = r[:, :2]
        newr[:, 0, 1] *= -1
        return newr

    @staticmethod
    def centre(data_2d):
        """center data according to each of the coordiante components"""
        return (data_2d.T - data_2d.mean(1)).T

    @staticmethod
    def centre_all(data):
        """center all data"""
        if data.ndim == 2:
            return Prob3dPose.centre(data)
        return (data.transpose(2, 0, 1) - data.mean(2)).transpose(1, 2, 0)

    @staticmethod
    def normalise_data(d2, weights):
        """Normalise data according to height"""

        # the joints with weight set to 0 should not be considered in the
        # normalisation process
        d2 = d2.reshape(d2.shape[0], -1, 2).transpose(0, 2, 1)
        idx_consider = weights[0, 0].astype(np.bool)
        if np.sum(weights[:, 0].sum(1) >= MIN_NUM_JOINTS) == 0:
            raise Exception(
                'Not enough 2D joints identified to generate 3D pose')
        d2[:, :, idx_consider] = Prob3dPose.centre_all(d2[:, :, idx_consider])

        # Height normalisation (2 meters)
        m2 = d2[:, 1, idx_consider].min(1) / 2.0
        m2 -= d2[:, 1, idx_consider].max(1) / 2.0
        crap = m2 == 0
        m2[crap] = 1.0
        d2[:, :, idx_consider] /= m2[:, np.newaxis, np.newaxis]
        return d2, m2

    @staticmethod
    def transform_joints(pose_2d, visible_joints):
        """
        Transform the set of joints according to what the probabilistic model
        expects as input.

        It returns the new set of joints of each of the people and the set of
        weights for the joints.
        """

        _H36M_ORDER = [8, 9, 10, 11, 12, 13, 1, 0, 5, 6, 7, 2, 3, 4]
        _W_POS = [1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16]

        def swap_xy(poses):
            tmp = np.copy(poses[:, :, 0])
            poses[:, :, 0] = poses[:, :, 1]
            poses[:, :, 1] = tmp
            return poses

        assert (pose_2d.ndim == 3)
        new_pose = pose_2d.copy()
        new_pose = swap_xy(new_pose)
        new_pose = new_pose[:, _H36M_ORDER]

        # defining weights according to occlusions
        weights = np.zeros((pose_2d.shape[0], 2, H36M_NUM_JOINTS))
        ordered_visibility = np.repeat(
            visible_joints[:, _H36M_ORDER, np.newaxis], 2, 2
        ).transpose([0, 2, 1])
        weights[:, :, _W_POS] = ordered_visibility
        return new_pose, weights

    def affine_estimate(self, w, depth_reg=0.085, weights=None, scale=10.0,
                        scale_mean=0.0016 * 1.8 * 1.2, scale_std=1.2 * 0,
                        cap_scale=-0.00129):
        """
        Quick switch to allow reconstruction at unknown scale returns a,r
        and scale
        """
        weights = np.zeros((0, 0, 0)) if weights is None else weights

        s = np.empty((self.sigma.shape[0], self.sigma.shape[1] + 4))  # e,y,x,z
        s[:, :4] = 10 ** -5  # Tiny but makes stuff well-posed
        s[:, 0] = scale_std
        s[:, 4:] = self.sigma
        s[:, 4:-1] *= scale

        e2 = np.zeros((self.e.shape[0], self.e.shape[
                      1] + 4, 3, self.e.shape[3]))
        e2[:, 1, 0] = 1.0
        e2[:, 2, 1] = 1.0
        e2[:, 3, 0] = 1.0
        # This makes the least_squares problem ill posed, as X,Z are
        # interchangable
        # Hence regularisation above to speed convergence and stop blow-up
        e2[:, 0] = self.mu
        e2[:, 4:] = self.e
        t_m = np.zeros_like(self.mu)

        res, a, r = pick_e(w, e2, t_m, self.cam, s, weights=weights,
                           interval=0.01, depth_reg=depth_reg,
                           scale_prior=scale_mean)

        scale = a[:, :, 0]
        reestimate = scale > cap_scale
        m = self.mu * cap_scale
        for i in range(scale.shape[0]):
            if reestimate[i].sum() > 0:
                ehat = e2[i:i + 1, 1:]
                mhat = m[i:i + 1]
                shat = s[i:i + 1, 1:]
                (res2, a2, r2) = pick_e(
                    w[reestimate[i]], ehat, mhat, self.cam, shat,
                    weights=weights[reestimate[i]],
                    interval=0.01, depth_reg=depth_reg,
                    scale_prior=scale_mean
                )
                res[i:i + 1, reestimate[i]] = res2
                a[i:i + 1, reestimate[i], 1:] = a2
                a[i:i + 1, reestimate[i], 0] = cap_scale
                r[i:i + 1, :, reestimate[i]] = r2
        scale = a[:, :, 0]
        a = a[:, :, 1:] / a[:, :, 0][:, :, np.newaxis]
        return res, e2[:, 1:], a, r, scale

    def better_rec(self, w, model, s=1, weights=1, damp_z=1):
        """Quick switch to allow reconstruction at unknown scale
        returns a,r and scale"""
        from numpy.core.umath_tests import matrix_multiply
        proj = matrix_multiply(self.cam[np.newaxis], model)
        proj[:, :2] = (proj[:, :2] * s + w * weights) / (s + weights)
        proj[:, 2] *= damp_z
        out = matrix_multiply(self.cam.T[np.newaxis], proj)
        return out

    def create_rec(self, w2, weights, res_weight=1):
        """Reconstruct 3D pose given a 2D pose"""
        _SIGMA_SCALING = 5.2

        res, e, a, r, scale = self.affine_estimate(
            w2, scale=_SIGMA_SCALING, weights=weights,
            depth_reg=0, cap_scale=-0.001, scale_mean=-0.003
        )

        remaining_dims = 3 * w2.shape[2] - e.shape[1]
        assert (remaining_dims >= 0)
        llambda = -np.log(self.sigma)
        lgdet = np.sum(llambda[:, :-1], 1) + llambda[:, -1] * remaining_dims
        score = (res * res_weight + lgdet[:, np.newaxis] * (scale ** 2))
        best = np.argmin(score, 0)
        index = np.arange(best.shape[0])
        a2 = a[best, index]
        r2 = r[best, :, index].T
        rec = Prob3dPose.build_and_rot_model(a2, e[best], self.mu[best], r2)
        rec *= -np.abs(scale[best, index])[:, np.newaxis, np.newaxis]

        rec = self.better_rec(w2, rec, 1, 1.55 * weights, 1) * -1
        rec = Prob3dPose.renorm_gt(rec)
        rec *= 0.97
        return rec

    def compute_3d(self, pose_2d, weights):
        """Reconstruct 3D poses given 2D estimations"""

        _J_POS = [1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16]
        _SCALE_3D = 1174.88312988

        if pose_2d.shape[1] != H36M_NUM_JOINTS:
            # need to call the linear regressor
            reg_joints = np.zeros(
                (pose_2d.shape[0], H36M_NUM_JOINTS, 2))
            for oid, singe_pose in enumerate(pose_2d):
                reg_joints[oid, _J_POS] = singe_pose

            norm_pose, _ = Prob3dPose.normalise_data(reg_joints, weights)
        else:
            norm_pose, _ = Prob3dPose.normalise_data(pose_2d, weights)

        pose_3d = self.create_rec(norm_pose, weights) * _SCALE_3D
        return pose_3d


_pose_estimator

In [ ]:
#from . import utils
import cv2
import numpy as np
import tensorflow as tf

import abc
ABC = abc.ABCMeta('ABC', (object,), {})

__all__ = [
    'PoseEstimatorInterface',
    'PoseEstimator'
]


class PoseEstimatorInterface(ABC):

    @abc.abstractmethod
    def initialise(self):
        pass

    @abc.abstractmethod
    def estimate(self, image):
        return

    @abc.abstractmethod
    def close(self):
        pass


class PoseEstimator(PoseEstimatorInterface):

    def __init__(self, image_size, session_path, prob_model_path):
        """Initialising the graph in tensorflow.
        INPUT:
            image_size: Size of the image in the format (w x h x 3)"""

        self.session = None
        self.poseLifting = Prob3dPose(prob_model_path)
        self.sess = -1
        self.orig_img_size = np.array(image_size)
        self.scale = INPUT_SIZE / (self.orig_img_size[0] * 1.0)
        self.img_size = np.round(
            self.orig_img_size * self.scale).astype(np.int32)
        self.image_in = None
        self.heatmap_person_large = None
        self.pose_image_in = None
        self.pose_centermap_in = None
        self.pred_2d_pose = None
        self.likelihoods = None
        self.session_path = session_path

    def initialise(self):
        """Load saved model in the graph
        INPUT:
            sess_path: path to the dir containing the tensorflow saved session
        OUTPUT:
            sess: tensorflow session"""

        tf.reset_default_graph()
        with tf.variable_scope('CPM'):
            # placeholders for person network
            self.image_in = tf.placeholder(
                tf.float32, [1, INPUT_SIZE, self.img_size[1], 3])

            heatmap_person = inference_person(self.image_in)

            self.heatmap_person_large = tf.image.resize_images(
                heatmap_person, [INPUT_SIZE, self.img_size[1]])

            # placeholders for pose network
            self.pose_image_in = tf.placeholder(
                tf.float32,
                [BATCH_SIZE, INPUT_SIZE, INPUT_SIZE, 3])

            self.pose_centermap_in = tf.placeholder(
                tf.float32,
                [BATCH_SIZE, INPUT_SIZE, INPUT_SIZE, 1])

            self.pred_2d_pose, self.likelihoods = inference_pose(
                self.pose_image_in, self.pose_centermap_in,
                INPUT_SIZE)

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        saver.restore(sess, self.session_path)

        self.session = sess

    def estimate(self, image):
        """
        Estimate 2d and 3d poses on the image.
        INPUT:
            image: RGB image in the format (w x h x 3)
            sess: tensorflow session
        OUTPUT:
            pose_2d: 2D pose for each of the people in the image in the format
            (num_ppl x num_joints x 2) visibility: vector containing a bool
            value for each joint representing the visibility of the joint in
            the image (could be due to occlusions or the joint is not in the
            image) pose_3d: 3D pose for each of the people in the image in the
            format (num_ppl x 3 x num_joints)
        """

        sess = self.session

        image = cv2.resize(image, (0, 0), fx=self.scale,
                           fy=self.scale, interpolation=cv2.INTER_CUBIC)
        b_image = np.array(image[np.newaxis] / 255.0 - 0.5, dtype=np.float32)

        hmap_person = sess.run(self.heatmap_person_large, {
                               self.image_in: b_image})

        hmap_person = np.squeeze(hmap_person)
        centers = detect_objects_heatmap(hmap_person)
        b_pose_image, b_pose_cmap = prepare_input_posenet(
            b_image[0], centers,
            [INPUT_SIZE, image.shape[1]],
            [INPUT_SIZE, INPUT_SIZE],
            batch_size=BATCH_SIZE)

        feed_dict = {
            self.pose_image_in: b_pose_image,
            self.pose_centermap_in: b_pose_cmap
        }

        # Estimate 2D poses
        pred_2d_pose, pred_likelihood = sess.run([self.pred_2d_pose,
                                                  self.likelihoods],
                                                 feed_dict)
        estimated_2d_pose, visibility = detect_parts_from_likelihoods(pred_2d_pose,
                                                                            centers,
                                                                            pred_likelihood)

        # Estimate 3D poses
        transformed_pose2d, weights = self.poseLifting.transform_joints(
            estimated_2d_pose.copy(), visibility)
        pose_3d = self.poseLifting.compute_3d(transformed_pose2d, weights)
        pose_2d = np.round(estimated_2d_pose / self.scale).astype(np.int32)

        return pose_2d, visibility, pose_3d

    def close(self):
        self.session.close()

#DEMO

running Demo

In [ ]:
!wget -o "https://drive.google.com/file/d/1NUHSl0HhSF2L3ywwUNuBscHzintNmGjQ/view?usp=sharing"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import __init__

'''from lifting import PoseEstimator
from lifting.utils import draw_limbs
from lifting.utils import plot_pose
'''
import cv2
import numpy
import matplotlib.pyplot as plt
from os.path import dirname, realpath

'''DIR_PATH = dirname(realpath(__file__))
PROJECT_PATH = realpath(DIR_PATH + '/..')
SAVED_SESSIONS_DIR = 'pose3d/data/saved_sessions'
SESSION_PATH = SAVED_SESSIONS_DIR + '/init_session/init'
PROB_MODEL_PATH = SAVED_SESSIONS_DIR + '/prob_model/prob_model_params.mat'
'''

IMAGE_FILE_PATH = 'test_image.png' #PROJECT_PATH + '/data/images/test_image.png'
SAVED_SESSIONS_DIR = '/content/gdrive/MyDrive/Pose3d/data/saved_sessions'
SESSION_PATH = '/content/init' #'/content/gdrive/MyDrive/Pose3d/data/saved_sessions/init_session/init'
PROB_MODEL_PATH = '/content/prob_model_params.mat' #'/content/gdrive/MyDrive/Pose3d/data/saved_sessions/prob_model/prob_model_params.mat'


def main():
    image = cv2.imread(IMAGE_FILE_PATH)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # conversion to rgb

    # create pose estimator
    image_size = image.shape

    pose_estimator = PoseEstimator(image_size, SESSION_PATH, PROB_MODEL_PATH)

    # load model
    pose_estimator.initialise()

    try:
        # estimation
        pose_2d, visibility, pose_3d = pose_estimator.estimate(image)

        # Show 2D and 3D poses
        display_results(image, pose_2d, visibility, pose_3d)
    except ValueError:
        print('No visible people in the image. Change CENTER_TR in packages/lifting/utils/config.py ...')

    # close model
    pose_estimator.close()


def display_results(in_image, data_2d, joint_visibility, data_3d):
    """Plot 2D and 3D poses for each of the people in the image."""
    plt.figure()
    draw_limbs(in_image, data_2d, joint_visibility)
    plt.imshow(in_image)
    plt.axis('off')

    # Show 3D poses
    for single_3D in data_3d:
        # or plot_pose(Prob3dPose.centre_all(single_3D))
        plot_pose(single_3D)

    plt.show()

if __name__ == '__main__':
    import sys
    sys.exit(main())
